In [1]:
import numpy as np 
import pandas as pd 
from ipynb.fs.full.funciones_FF import feature_RatioDayMoments, feature_Basicos, feature_Basicos_Ult_7d

from ipynb.fs.full.funciones_CF import qty_adoptions, qty_days_last_adoption
from ipynb.fs.full.funciones_CF import qty_days_last_visit, qty_days_last_visit_vs_adoption
from ipynb.fs.full.funciones_CF import ratio_page_qty_adopciones

from ipynb.fs.full.funciones_LB import ratio_page_all
from ipynb.fs.full.funciones_LB import ratio_ccategory_all
from ipynb.fs.full.funciones_LB import ratio_cctop_all
from ipynb.fs.full.funciones_LB import ratio_ccbottom_all
from ipynb.fs.full.funciones_LB import ratio_site_id_all
from ipynb.fs.full.funciones_LB import ratio_mobile_device_all
from ipynb.fs.full.funciones_LB import ratio_connection_speed_all
from ipynb.fs.full.funciones_LB import ratio_on_site_search_term_all

# from ipynb.fs.full.funciones_LB import ratio_adoption_page_all, ratio_adoption_ccategory_all 
# from ipynb.fs.full.funciones_LB import ratio_adoption_cctop_all, ratio_adoption_ccbottom_all
# from ipynb.fs.full.funciones_LB import ratio_adoption_site_id_all, ratio_adoption_mobile_device_all
# from ipynb.fs.full.funciones_LB import ratio_adoption_connection_speed_all, ratio_adoption_on_site_search_term_all,
# from ipynb.fs.full.funciones_LB import ratio_page_all, ratio_category_all, ratio_cctop_all, ratio_ccbottom_all,
# from ipynb.fs.full.funciones_LB import ratio_site_id_all, ratio_mobile_device_all
# from ipynb.fs.full.funciones_LB import ratio_connction_speed_all, ratio_on_site_search_term_al


In [2]:
# Para que levante las funciones que se van actualizando en los otros notebooks
%load_ext autoreload
%autoreload 2

In [3]:
# Alternativa (peor) al import ipynb.fs.full
#%run funciones_FF.ipynb
#%run funciones_CF.ipynb

# Seteo del mes snapshot

In [4]:
mes_snapshot = 9  #9 para train 12 para scoreo

# Lectura de los csv

In [5]:
df_pageviews = pd.read_csv("./data/pageviews.csv",
                   parse_dates=["FEC_EVENT"])
df_pageviews_complemento = pd.read_csv("./data/pageviews_complemento.csv",
                   parse_dates=["FEC_EVENT"])
df_pageviews = pd.concat([df_pageviews, df_pageviews_complemento],ignore_index=True,sort=False).drop_duplicates().reset_index(drop=True)
df_devicedata = pd.read_csv("./data/device_data.csv",
                   parse_dates=["FEC_EVENT"]).drop_duplicates().reset_index(drop=True)
df_conversiones = pd.read_csv("./data/conversiones.csv").drop_duplicates().reset_index(drop=True)

In [6]:
#Se eliminan los registros que no deben ser considerados.
df_pageviews = df_pageviews[df_pageviews['FEC_EVENT'].dt.month <= mes_snapshot]
df_pageviews = df_pageviews[df_pageviews['FEC_EVENT'].dt.month > mes_snapshot - 9]

df_devicedata = df_devicedata[df_devicedata['FEC_EVENT'].dt.month <= mes_snapshot]
df_devicedata = df_devicedata[df_devicedata['FEC_EVENT'].dt.month > mes_snapshot - 9]

# Creación del universo

In [7]:
def universo_train(df):
    universo = pd.DataFrame()
    universo['user_id'] = df.USER_ID.unique()
    universo['snapshot_mes'] = mes_snapshot
    return universo

In [8]:
universo = universo_train(df_pageviews)
universo.head(10)

,user_id,snapshot_mes
0,0,9
1,1,9
2,2,9
3,3,9
4,4,9
5,5,9
6,6,9
7,7,9
8,8,9
9,9,9


# Features

In [9]:
# def feature_X(df):

In [10]:
# features mix
# ratio_adoption_page_all(universo, df_pageviews, df_adoption, mes_snapshot)
# ratio_adoption_ccategory_all(universo, df_pageviews, df_adoption, mes_snapshot)
# ratio_adoption_cctop_all(universo, df_pageviews, df_adoption, mes_snapshot)
# ratio_adoption_ccbottom_all(universo, df_pageviews, df_adoption, mes_snapshot)
# ratio_adoption_site_id_all(universo, df_pageviews, df_adoption, mes_snapshot)
# ratio_adoption_mobile_device_all(universo, df_pageviews, df_adoption, mes_snapshot)
# ratio_adoption_connection_speed_all(universo, df_pageviews, df_adoption, mes_snapshot)

In [11]:
#features de pageviews
# ratio_page_all(universo, df_pageviews, mes_snapshot)
# ratio_category_all(universo, df_pageviews, mes_snapshot)
# ratio_cctop_all(universo, df_pageviews, mes_snapshot)
# ratio_ccbottom_all(universo, df_pageviews, mes_snapshot)
# ratio_site_id_all(universo, df_pageviews, mes_snapshot)
# on_site_search_term_user(universo, df_pageviews, mes_snapshot)

In [12]:
#features de device
# ratio_mobile_device_all(universo, df_devicedata, mes_snapshot)
# ratio_connction_speed_all(universo, df_devicedata, mes_snapshot)
# ratio_on_site_search_term_all(universo, df_devicedata, mes_snapshot)
# ratio_adoption_on_site_search_term_all(universo, df_devicedata, mes_snapshot)

In [13]:
# FF methods
df_aux = df_pageviews.copy()
universo = feature_RatioDayMoments(universo, df_aux)
# universo = feature_Basicos(universo,df_pageviews) 
# universo = feature_Basicos_Ult_7d(universo,df_pageviews)

In [15]:
# CF methods
df_aux = df_pageviews.copy()
universo = qty_adoptions(universo, df_conversiones)
universo = qty_days_last_adoption(universo, df_conversiones)
universo = qty_days_last_visit(universo, df_pageviews)
universo = qty_days_last_visit_vs_adoption(universo, df_aux, df_conversiones)
universo = ratio_page_qty_adopciones(universo, df_pageviews, df_conversiones)

ValueError: Wrong number of items passed 2, placement implies 1

In [ ]:
universo = ratio_page_all(universo, df_pageviews, mes_snapshot)
universo = ratio_ccategory_all(universo, df_pageviews, mes_snapshot)
universo = ratio_cctop_all(universo, df_pageviews, mes_snapshot)
universo = ratio_ccbottom_all(universo, df_pageviews, mes_snapshot)
universo = ratio_site_id_all(universo, df_pageviews, mes_snapshot)
universo = ratio_mobile_device_all(universo, df_devicedata, mes_snapshot)
universo = ratio_connection_speed_all(universo, df_devicedata, mes_snapshot)
# ratio_on_site_search_term_all(universo, df_devicedata, mes_snapshot)

In [ ]:
# check universo
universo.head(10)  

# Target

In [ ]:
def target(df):
    df = pd.merge(df, df_conversiones, how='left', left_on=['user_id'], right_on=['USER_ID'])
    df.mes.fillna(value= 99, inplace=True)
    df = df.astype({'mes': 'int32'}) 
    
    df['target'] = 0
    df.loc[df.mes == df.snapshot_mes +1, 'target'] = 1
    df.loc[df.mes == df.snapshot_mes +2, 'target'] = 1
    df.loc[df.mes == df.snapshot_mes +3, 'target'] = 1

    return (df.groupby(['user_id', 'snapshot_mes'])['target'].max()).to_frame().reset_index()

In [ ]:
target_universo = target(universo)
universo = pd.merge(universo, target_universo, how='left', left_on=['user_id', 'snapshot_mes'], right_on=['user_id','snapshot_mes'])

In [ ]:
universo.to_csv(r'dataset.csv', index = None, header=True)

In [ ]:
df_pageviews.to_csv(r'df_pageviews.csv', index = None, header=True)

In [ ]:
df_conversiones.to_csv(r'df_conversiones.csv', index = None, header=True)

In [ ]:
df_devicedata.to_csv(r'df_devicedata.csv', index = None, header=True)